In [ ]:
import time
import psutil
from collections import deque

from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure
from bokeh.models import Range1d

from nupic.data.inference_shifter import InferenceShifter
from nupic.frameworks.opf.modelfactory import ModelFactory

import model_params

SECONDS_PER_STEP = 2
WINDOW = 60

output_notebook()
fig = figure(title='CPU prediction example', y_range=(0,100))
fig.xaxis.axis_label = "time [s]"
fig.yaxis.axis_label = "CPU usage [%]"
# Keep the last WINDOW predicted and actual values for plotting.
actual_history = deque([0.0] * WINDOW, maxlen=60)
prediction_history = deque([0.0] * WINDOW, maxlen=60)

# Initialize the plot lines that we will update with each new record.
actual_line = fig.line(range(WINDOW), actual_history, color='blue', legend='History', line_width=5)
prediction_line = fig.line(range(WINDOW), prediction_history, color='red', legend='Prediction', line_width=3)


In [ ]:
def run_cpu():
  """Poll CPU usage, make predictions, and plot the results. Runs forever."""
  # Create the model for predicting CPU usage.
  model = ModelFactory.create(model_params.MODEL_PARAMS)
  model.enableInference({'predictedField': 'cpu'})
  # The shifter will align prediction and actual values.
  shifter = InferenceShifter()

  while True:
    s = time.time()

    # Get the CPU usage.
    cpu = psutil.cpu_percent()

    # Run the input through the model and shift the resulting prediction.
    modelInput = {'cpu': cpu}
    result = shifter.shift(model.run(modelInput))

    # Update the trailing predicted and actual value deques.
    inference = result.inferences['multiStepBestPredictions'][5]
    if inference is not None:
      actual_history.append(result.rawInput['cpu'])
      prediction_history.append(inference)
      actual_line.data_source.data["y"] = actual_history
      prediction_line.data_source.data["y"] = prediction_history
      push_notebook(handle=h)
    time.sleep(SECONDS_PER_STEP)


In [ ]:
h = show(fig, notebook_handle=True)

In [ ]:
run_cpu()

In [ ]:
def run_num():
  """Fake CPU with an ever increasing integer."""
  # Create the model for predicting CPU usage.
  model = ModelFactory.create(model_params.MODEL_PARAMS)
  model.enableInference({'predictedField': 'cpu'})
  # The shifter will align prediction and actual values.
  shifter = InferenceShifter()

  for i in range(100):
    s = time.time()

    # Get the CPU usage.
    cpu = i

    # Run the input through the model and shift the resulting prediction.
    modelInput = {'cpu': cpu}
    result = shifter.shift(model.run(modelInput))

    # Update the trailing predicted and actual value deques.
    inference = result.inferences['multiStepBestPredictions'][5]
    if inference is not None:
      actual_history.append(result.rawInput['cpu'])
      prediction_history.append(inference)
      actual_line.data_source.data["y"] = actual_history
      prediction_line.data_source.data["y"] = prediction_history
      push_notebook(handle=h)
    time.sleep(SECONDS_PER_STEP)

In [ ]:
run_num()